In [ ]:
#| default_exp gnncoalv2

In [ ]:
#| export
import torch
import torch.nn as nn
from torch_geometric.nn.dense.dense_gcn_conv import DenseGCNConv
from math import ceil
import torch.nn.functional as F
from torch_geometric.nn import dense_diff_pool
from x_transformers import Encoder

import random
import numpy as np
import torch

In [ ]:
#| export

def seed_everything(seed=42):
    """Seed all random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # Ensure deterministic behavior (this can slow down training, so you might not always want it)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example usage:
# seed_everything(42)


In [ ]:
#| export

class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, linearize=True):
        super(GNN, self).__init__()

        self.convs = nn.ModuleList([
            DenseGCNConv(in_channels, hidden_channels, improved=True),
            DenseGCNConv(hidden_channels, hidden_channels, improved=True),
            DenseGCNConv(hidden_channels, out_channels, improved=True)
        ])

        self.norms = nn.ModuleList([
            nn.LayerNorm(hidden_channels),
            nn.LayerNorm(hidden_channels),
            nn.LayerNorm(out_channels)
        ])

        self.linearize = linearize
        if self.linearize: self.linear = nn.Linear(2 * hidden_channels + out_channels, out_channels)
        else: self.linear = None
        

    def forward(self, x, adj):
        xs = []
        for conv, norm in zip(self.convs, self.norms):
            x = F.relu(conv(x, adj))
            x = norm(x)
            xs.append(x)
        x = torch.cat(xs, dim=-1)
        
        if self.linearize: return F.relu(self.linear(x))
        else: return x

In [ ]:
#| export

class DiffPoolConvolutionPart(nn.Module):
    def __init__(self, max_nodes, num_features, num_hidden=64):
        super().__init__()

        self.layers = []
        num_nodes = ceil(0.3 * max_nodes)
        
        for _ in range(2):  
            self.layers.append({
                'pool': GNN(num_features, num_hidden, num_nodes),
                'embed': GNN(num_features, num_hidden, num_hidden, linearize=False)
            })
            num_features = 3 * num_hidden
            num_nodes = ceil(0.3 * num_nodes)
            
        self.gnn3_embed = GNN(3 * num_hidden, num_hidden, num_hidden, linearize=False)

    def forward(self, x, adj, batch_size, num_trees=500):
        l_total, e_total = 0, 0
        for layer in self.layers:
            s = layer['pool'](x, adj)
            x = layer['embed'](x, adj)
            x, adj, l, e = dense_diff_pool(x, adj, s)
            l_total += l
            e_total += e
            
        x = self.gnn3_embed(x, adj)
        x = x.mean(dim=1)
        x = x.view(batch_size, num_trees, x.shape[-1])
        
        return x, l_total, e_total
        
#model = GNNcoalV2(max_nodes=19, num_features=60, num_hidden=64,  out_channels=60)

In [ ]:
#| export

class MiniDenseNet(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(MiniDenseNet, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#| export

def mean_over_n(emb, n=10):
    # Unfold the tensor along dimension 1, using size n and step n
    unfolded = emb.unfold(1, n, n)
    # Compute the mean over the new last dimension
    mean_values = unfolded.mean(dim=-1)
    return mean_values

In [ ]:
#| export

class GNNcoalV2(nn.Module):
    def __init__(self, max_nodes, num_features, num_hidden=64, out_dim=60,
                 enc_heads = 4, enc_depth=8,
                ):
        super().__init__()

        self.conv = DiffPoolConvolutionPart(max_nodes=max_nodes,
                                            num_features=num_features,
                                            num_hidden=num_hidden)
        enc_dim = num_hidden * 3
        self.encoder = Encoder(dim=enc_dim, depth=enc_depth, heads=enc_heads,
                               ff_glu=True, residual_attn=True, flash=True)

        self.demography_head = MiniDenseNet(enc_dim, enc_dim//2, out_dim)
        self.alpha_head = MiniDenseNet(enc_dim, enc_dim//2, out_dim)
        

        

    def forward(self, x, adj, batch_size, num_trees=500, num_trees_for_alpha=10):
        emb, l_total, e_total = self.conv(x, adj, batch_size, num_trees)
        emb = self.encoder(emb)    
        
        demo_emb = emb.mean(dim=1)
        alpha_emb = mean_over_n(emb, num_trees_for_alpha)

        demography_estimate = self.demography_head(demo_emb)
        seq_alpha_estimate = self.alpha_head(alpha_emb)
        
        return demography_estimate, seq_alpha_estimate, l_total, e_total
        

# model = GNNcoalV2(max_nodes=19, num_features=60, num_hidden=64,  out_dim=60,
#                   enc_heads=4, enc_depth=8,
#                  )

In [ ]:
#sum([p.numel() for p in model.parameters() if p.requires_grad == True])

In [ ]:
# import msprime
# import seaborn as sns
# import matplotlib.pyplot as plt


# tss = []

# for i in range(32):
#     ts = msprime.sim_ancestry(samples=10, population_size=10_000,
#                               sequence_length=2e6, ploidy=1, recombination_rate=1e-8)
#     tss.append(ts)

In [ ]:
# def ts_to_data_objects(ts, num_trees=500, num_embedding=60, y=None):
#     max_num_nodes = 2 * ts.num_samples - 1
#     data_objects = []
#     for i, tree in enumerate(ts.trees()):
#         if i < num_trees:
#             data = from_networkx(nx.Graph(tree.as_dict_of_dicts()))
#             data.edge_weight = data.branch_length 
#             data.branch_length  = None
#             data.x = torch.eye(max_num_nodes, num_embedding)
#             data.x[data.num_nodes:] = torch.zeros(num_embedding)
#             data.num_nodes = max_num_nodes
#             data.y = y
#             data_objects.append(data)
#     return data_objects

In [ ]:
# data_objects = []
# for ts in tss:
#     i_data_object = ts_to_data_objects(ts)
#     data_objects += i_data_object

In [ ]:
# import torch
# import networkx as nx
# from torch_geometric.loader import DataLoader
# from torch_geometric.utils.convert import from_networkx
# from torch_geometric.utils.to_dense_adj import to_dense_adj
# from torch_geometric.utils.to_dense_batch import to_dense_batch


# bs = 8
# dl = DataLoader(data_objects, batch_size=500*bs)
# data_batch = next(iter(dl))
# batch = data_batch
# adj = to_dense_adj(batch.edge_index, batch=batch.batch, edge_attr=batch.edge_weight, max_num_nodes=None)
# x, _ = to_dense_batch(batch.x, batch.batch)

In [ ]:
#demography, alpha_values, l_total, e_total = model(x, adj, batch_size=8)